# 📊 Análise Exploratória do Censo e Produção Científica

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sqlite3
import scipy.stats as stats
import statsmodels.api as sm

pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

In [ ]:
from scripts.limpeza import carregar_e_filtrar

base_path = 'data/'
df_cursos, df_ies = carregar_e_filtrar(base_path)

print(df_cursos.head())
print(df_ies.head())

## Cursos por UF

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(data=df_cursos, x='SG_UF', order=df_cursos['SG_UF'].value_counts().index)
plt.title('Distribuição de Cursos por UF')
plt.xticks(rotation=45)
plt.show()

## Cursos por Categoria Administrativa

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df_cursos, x='CATEGORIA_ADMINISTRATIVA', order=df_cursos['CATEGORIA_ADMINISTRATIVA'].value_counts().index)
plt.title('Cursos por Categoria Administrativa')
plt.show()

## Cursos por Organização Acadêmica

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df_cursos, x='ORGANIZACAO_ACADEMICA', order=df_cursos['ORGANIZACAO_ACADEMICA'].value_counts().index)
plt.title('Cursos por Organização Acadêmica')
plt.show()

## Heatmap: Cursos por UF vs Organização Acadêmica

In [ ]:
pivot = df_cursos.pivot_table(index='SG_UF', columns='ORGANIZACAO_ACADEMICA',
                              values='NO_CURSO', aggfunc='count', fill_value=0)
plt.figure(figsize=(15,10))
sns.heatmap(pivot, annot=True, fmt='d', cmap='YlGnBu')
plt.title('Heatmap: Cursos por UF vs Organização Acadêmica')
plt.show()

## Consulta SQL Relacional

In [ ]:
conn = sqlite3.connect('database.db')

query = """
SELECT 
    cursos.SG_UF, 
    COUNT(*) as qtd_cursos,
    (SELECT COUNT(*) FROM bolsistas WHERE bolsistas."Sigla UF Destino" = cursos.SG_UF) as qtd_bolsistas
FROM cursos
GROUP BY cursos.SG_UF
ORDER BY qtd_cursos DESC
"""

df_sql = pd.read_sql(query, conn)
conn.close()

print(df_sql)

## Visualização da Consulta SQL

In [ ]:
fig = px.bar(df_sql.melt(id_vars='SG_UF', value_vars=['qtd_cursos', 'qtd_bolsistas']),
             x='SG_UF', y='value', color='variable',
             barmode='group',
             labels={'value': 'Quantidade', 'variable': 'Indicador'},
             text_auto=True,
             title='Comparação: Cursos vs Bolsistas por UF (via SQL)',
             width=1000, height=500)
fig.show()

## Análise Estatística: Correlação

In [ ]:
corr, p_value = stats.pearsonr(df_sql['qtd_cursos'], df_sql['qtd_bolsistas'])
print(f'Correlação Pearson: {corr:.2f} (p-valor: {p_value:.4f})')

## Regressão Linear

In [ ]:
X = df_sql['qtd_cursos']
y = df_sql['qtd_bolsistas']

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

## Conclusões e Recomendações

- Maior concentração de cursos em algumas UFs.
- Correlação positiva entre cursos e bolsistas CNPq.
- A regressão confirma relação direta.

**Recomendações:**

- Fomentar bolsas onde há menor densidade.
- Incentivar instituições privadas a investir mais em pesquisa.
- Melhorar a integração entre bases públicas.